#

In [96]:
import pandas as pd

In [97]:
acs_pums_df = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/raw_data/national_level/ACS-PUMS-National.csv')
# /content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/raw_data/Minnesota - ACS PUMS - Five-Years/psam_p27.csv
df_lookup = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/reference_data/Pums Ref Data - Sheet1.csv')
age_lookup = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/reference_data/ACS Age bins - Ref - Sheet1.csv')

In [98]:
df_lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   VARIABLE_NAME  36 non-null     object
 1   CODE           36 non-null     object
 2   VALUE          36 non-null     object
dtypes: object(3)
memory usage: 992.0+ bytes

In [99]:
df_lookup.head(5)

In [100]:
green_jobs_df = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/processed_data/Green Enhanced Skills Occupations - All - Sheet1.csv')

green_jobs_df.head()

In [101]:
lookup_dict = df_lookup.groupby('VARIABLE_NAME').apply(lambda x: x.set_index('CODE')['VALUE'].to_dict()).to_dict()

# Function to replace codes with values based on the mapping
def replace_codes(df, column, code_to_value):
    if column in code_to_value:
      df[column] = df[column].map(code_to_value[column]).fillna(df[column])
    return df

# lookup_dict

In [102]:
acs_pums_df = acs_pums_df.astype(str)
age_lookup['Age Group'] = age_lookup['Age Group'].astype(str)
acs_pums_df['tabulate'] = acs_pums_df['tabulate'].astype(int)

In [103]:
age_lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Age Group  6 non-null      object
 1   Min        6 non-null      int64 
 2   Max        6 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 272.0+ bytes

In [104]:
# Apply the replacement for each column in df_psam_p27
for column in acs_pums_df.columns:
    acs_pums_df = replace_codes(acs_pums_df, column, lookup_dict)

In [105]:
acs_pums_df.head(100)

100 rows × 6 columns

In [106]:
acs_pums_age_bins_merged_df = pd.merge(acs_pums_df, age_lookup, left_on='AGEGROUPS', right_on='Age Group', how='left')
acs_pums_age_bins_merged_df.head()

In [107]:
green_jobs_df_selected = green_jobs_df[['O*NET-SOC Title', 'O*NET-SOC Code']]
green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.split('.').str[0]
green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.replace('-', '')
# Get unique rows based on the selected columns
green_jobs_df_unique = green_jobs_df_selected.drop_duplicates()


# Display the first few rows of the unique DataFrame
print(green_jobs_df_unique.head(10))

                            O*NET-SOC Title O*NET-SOC Code
0                     Power Plant Operators         518013
1       Continuous Mining Machine Operators         475041
4         Storage and Distribution Managers         113071
5   Maintenance and Repair Workers, General         499042
7                      Electrical Engineers         172071
8                                Machinists         514041
10                     Mechanical Engineers         172141
11                      Sheet Metal Workers         472211
21  Nuclear Equipment Operation Technicians         194051
22          Nuclear Power Reactor Operators         518011

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.split('.').str[0]
<ipython-input-107-bf50efa47340>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.replace('-', '')

In [108]:
green_jobs_df_unique = green_jobs_df_unique.groupby('O*NET-SOC Code').agg({'O*NET-SOC Title': 'first'}).reset_index()

In [109]:
green_jobs_acs_df_merged = pd.merge(acs_pums_age_bins_merged_df, green_jobs_df_unique, left_on='SOCP', right_on='O*NET-SOC Code', how='left')

# Display the first few rows of the merged DataFrame
print(green_jobs_acs_df_merged.head())

   tabulate     SEX                  RAC1P          SCHL    SOCP AGEGROUPS  \
0         0    Male  American Indian alone  Kindergarten  339021         1   
1         0  Female  American Indian alone  Kindergarten  339021         1   
2         0    Male            White alone  Kindergarten  339021         1   
3         0  Female            White alone  Kindergarten  339021         1   
4         0    Male  Some Other Race alone  Kindergarten  339021         1   

  Age Group  Min  Max O*NET-SOC Code O*NET-SOC Title  
0         1   16   24            NaN             NaN  
1         1   16   24            NaN             NaN  
2         1   16   24            NaN             NaN  
3         1   16   24            NaN             NaN  
4         1   16   24            NaN             NaN  

In [110]:
green_jobs_acs_df_merged_filtered = green_jobs_acs_df_merged.dropna(subset=['O*NET-SOC Title', 'O*NET-SOC Code'])
len(green_jobs_acs_df_merged_filtered)
len(green_jobs_acs_df_merged)

1433700

In [111]:
green_jobs_acs_df_merged_filtered.head()

In [112]:
green_jobs_acs_df_merged_filtered[green_jobs_acs_df_merged_filtered['tabulate'] > 0].head(100)

100 rows × 11 columns

In [113]:
green_jobs_acs_df_merged['Green Job Flag'] = green_jobs_acs_df_merged['O*NET-SOC Code'].notnull().astype(int)

In [114]:
# 78300
flag_counts = green_jobs_acs_df_merged['Green Job Flag'].value_counts()

# Display the counts
print(flag_counts)

Green Job Flag
0    1355400
1      78300
Name: count, dtype: int64

In [115]:
green_jobs_acs_df_merged.to_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/processed_data/National_ACS_All_Jobs.csv', index=False)